In [ ]:
!pip install datasets trl
!pip uninstall -y bitsandbytes
!pip install --upgrade bitsandbytes
!pip install --upgrade transformers accelerate
!nvidia-smi
!pip install accelerate
!accelerate config
!pip install flash-attn --no-build-isolation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!apt-get update
!apt-get install -y tesseract-ocr
!pip install pytesseract
!tesseract -v

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,321 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [68.9 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,664 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,688 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
import torch
import random
import os

###Carga modelo

In [ ]:
import pytesseract
import spacy
import re
import torch
import requests
import json
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from google.colab import files

# Configuración Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'
nlp = spacy.load("en_core_web_sm")

# Cargar modelo entrenadoimport torch
# Definir el dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

# Nombre del modelo en Hugging Face
model_name = "CasiAC/deepseek-ciberseguridad-full-lora"

# Configuración de quantización en 4 bits para optimizar memoria
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Carga el modelo en 4 bits en lugar de 8 bits
    bnb_4bit_compute_dtype=torch.float16,  # Usa FP16 para cálculos
    bnb_4bit_use_double_quant=True,  # Habilita doble cuantización para optimización
    bnb_4bit_quant_type="nf4"  # Cuantización NF4 (recomendada por Hugging Face)
)

# Cargar el modelo con cuantización en 4 bits
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",  # Distribuye el modelo entre CPU/GPU automáticamente
    trust_remote_code=True
)

# Cargar el tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

print("✅ Modelo cargado en 4 bits con éxito 🚀")

Usando dispositivo: cuda


adapter_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/371 [00:00<?, ?B/s]

✅ Modelo cargado en 4 bits con éxito 🚀


### Pipeline modelo + OCR

In [ ]:
from PIL import Image, ImageEnhance
import pytesseract
import re
import requests
import json

# 🔹 API Key de VirusTotal
API_KEY = "06858db9f480b4aba21a5831457a9b919b1f9014e6f8872ee1f4f7d1a029197c"
HEADERS = {"x-apikey": API_KEY}

def preprocesar_imagen(imagen):
    """Convierte la imagen a escala de grises y mejora el contraste."""
    try:
        image = Image.open(imagen)
        image = image.convert("L")
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(2.0)
        return image
    except Exception as e:
        return None

def extraer_texto_img(imagen):
    """Extrae texto de una imagen tras preprocesarla."""
    try:
        image = preprocesar_imagen(imagen)
        if image is None:
            return "Error al procesar la imagen"
        # Extraer texto con pytesseract
        texto_extraido = pytesseract.image_to_string(image)
        return limpiar_texto(texto_extraido)
    except Exception as e:
        return f"Error al procesar la imagen: {str(e)}"

def consultar_ip(ip):
    """Consulta una IP en VirusTotal y evalúa si es segura o maliciosa."""
    url = f"https://www.virustotal.com/api/v3/ip_addresses/{ip}"
    response = requests.get(url, headers=HEADERS)

    if response.status_code == 200:
        data = response.json()
        stats = data["data"]["attributes"]["last_analysis_stats"]
        malicious = stats.get("malicious", 0)
        harmless = stats.get("harmless", 0)

        if malicious > 0:
            veredicto = f"❌ La IP {ip} ha sido reportada como **maliciosa** en {malicious} análisis."
        else:
            veredicto = f"✅ La IP {ip} parece **segura**, sin reportes de actividad maliciosa."

        return {
            "IP": ip,
            "Veredicto": veredicto,
            "Análisis": stats
        }
    return {"error": f"Error en la consulta: {response.status_code}"}

def consultar_url(url):
    """Consulta una URL en VirusTotal y evalúa si es segura o maliciosa."""
    scan_url = "https://www.virustotal.com/api/v3/urls"
    response = requests.post(scan_url, headers=HEADERS, data={"url": url})

    if response.status_code == 200:
        analysis_id = response.json()["data"]["id"]
        result_url = f"https://www.virustotal.com/api/v3/analyses/{analysis_id}"
        result_response = requests.get(result_url, headers=HEADERS)

        if result_response.status_code == 200:
            data = result_response.json()
            stats = data["data"]["attributes"]["stats"]
            malicious = stats.get("malicious", 0)
            harmless = stats.get("harmless", 0)

            if malicious > 0:
                veredicto = f"❌ La URL {url} ha sido **marcada como maliciosa** en {malicious} análisis."
            else:
                veredicto = f"✅ La URL {url} parece **segura**, sin reportes de actividad maliciosa."

            return {
                "URL": url,
                "Veredicto": veredicto,
                "Análisis": stats
            }

    return {"error": f"Error en la consulta: {response.status_code}"}

def analizar_prompt(prompt):
    """Detecta si el prompt contiene una IP o URL y consulta VirusTotal si es necesario."""

    if isinstance(prompt, str) and (prompt.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.webp', '.heic'))):
        print(f"🔍 Detectada imagen: {prompt}")
        texto_extraido = extraer_texto_img(prompt)
        print(f"Texto extraído de la imagen: {texto_extraido}")

        return analizar_con_modelo(texto_extraido)

    ip_pattern = r"\b(?:\d{1,3}\.){3}\d{1,3}\b"
    url_pattern = r'(https?://[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:/[^ \n]*)?|www\.[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:/[^ \n]*)?|[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:/[^ \n]*)?)'

    ip_match = re.search(ip_pattern, prompt)
    url_match = re.search(url_pattern, prompt)

    if ip_match:
        ip = ip_match.group()
        print(f"🔍 Detectada IP en el prompt: {ip}")
        return consultar_ip(ip)

    if url_match:
        url = url_match.group()
        print(f"🔍 Detectada URL en el prompt: {url}")
        return consultar_url(url)

    return None  # No se detectó ninguna IP o URL

def generar_respuesta(prompt):
    """Genera una respuesta con el modelo o consulta VirusTotal si es necesario."""

    resultado_api = analizar_prompt(prompt)

    if resultado_api:
        return json.dumps(resultado_api, indent=4, ensure_ascii=False)  # Respuesta en JSON

    # Tokenizar el prompt y generar la respuesta
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)

    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=300,
        temperature=0.3,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.5,
        pad_token_id=tokenizer.eos_token_id,
    )

    # Decodificar y limpiar la salida
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return response.replace(prompt, "").strip()

def analizar_con_modelo(texto_extraido):
    """Pasa el texto al modelo para que lo analice en busca de señales claras de phishing."""

    texto_limpio = limpiar_texto(texto_extraido)

    system_prompt = """
      You are a highly specialized AI in cybersecurity. Your primary task is to analyze messages and detect phishing attempts with precision.

      🔹 STRICT RULES:
      1. Do not invent information. You must base your response ONLY on the given text.
      2. Do not provide general explanations unless explicitly asked.
      3. Be concise and precise. Your response must be short and strictly relevant.
      4. Use formal cybersecurity language and avoid assumptions.
      5. Output must always start with either "Phishing" or "Not Phishing", followed by a brief explanation of why.

      🔹 HOW TO ANALYZE A MESSAGE FOR PHISHING:
      - Urgency: Does the message pressure the user to act quickly?
      - Suspicious Links: Does it contain shortened or untrusted links?
      - Requests for Personal Information: Is the user asked to provide passwords or sensitive data?
      - Errors or Inconsistencies: Are there grammar mistakes, unnatural tone, or unusual sender details?

      🔹 RESPONSE FORMAT (STRICT):
      ```
      Phishing or Not Phishing
      Explanation: [Concise justification, mentioning phishing indicators if present]
      ```
      ---
    """

    prompt_modelo = f"""
  	  Analyze the following message and determine whether it is a **phishing attempt** based on the criteria defined in the system instructions.

      REMEMBER: Your response must strictly follow the required format. Do not repeat instructions or add unnecessary details.

      MESSAGE TO EVALUATE:
      {texto_extraido}
    """

    # Tokenizamos el prompt y lo pasamos al modelo
    inputs = tokenizer(system_prompt + prompt_modelo, return_tensors="pt", padding=True, truncation=True).to(device)

    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=300,  # Limitar los tokens de la respuesta
        temperature=0.15,  # Controlar la aleatoriedad
        do_sample=True,  # Sin aleatorización
        top_k=10,  # Menos diversidad
        top_p=0.7,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id,
    )

    # Decodificar y limpiar la salida
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    if "</think>" in response:
        user_answer = response.split("</think>", 1)[-1].strip()

    return user_answer



def limpiar_texto(texto):
    """Limpia el texto extraído de caracteres no deseados."""
    texto = texto.strip()
    texto = re.sub(r'\s+', ' ', texto)  # Reemplazar espacios extra
    texto = texto.replace("\n", " ").replace("\r", "")  # Eliminar saltos de línea
    return texto



In [ ]:
prompt_usuario_4 = 'image6.png'
respuesta_4 = generar_respuesta(prompt_usuario_4)
print(f"\n🔹 Respuesta para imagen:\n{respuesta_4}")

🔍 Detectada imagen: image6.png
Texto extraído de la imagen: Sent on: Friday, June 23, 2023 11:31:12 AM To: Subject: YOUR ACCOUNT IS AT RISK!! Dear Valued User , We received a request from you to terminate your Office 365 email due to a dual college/universities account. This process has begun by our administrator. If you did not authorize this action and you have no knowledge of it, you are advised to re-verify your account. Please give us 24 hours to terminate your account if you initiated the request. Failure to re-verify will result in the closure of your account and you will lose all of my files on these 365 accounts. If this request was made accidentally and you have no knowledge of it, you are advised to copy and paste the URL Below into the address bar of your web browser to fill in the form. cutt.ly/OwtNi6KO Failure to Verify will result in the closure of your account. lowa State University IT Helpdesk All Right Reserved.

🔹 Respuesta para imagen:
"```json\nPhishing\nExplanatio